# Flax Basics

This notebook will walk you through the following workflow:

*   Instantiating a model from Flax built-in layers or third-party models.
*   Initializing parameters of the model and manually written training.
*   Using optimizers provided by Flax to ease training.
*   Serialization of parameters and other objects.
*   Creating your own models and managing state.

## Setting up our environment

Here we provide the code needed to set up the environment for our notebook.

In [1]:
# # Install the latest JAXlib version.
# !pip install --upgrade -q pip jax jaxlib
# # Install Flax at head:
# !pip install --upgrade -q git+https://github.com/google/flax.git

In [2]:
import jax
from typing import Any, Callable, Sequence, Optional
import jax
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze
from flax import linen as nn ## nn exression

## Linear regression with Flax

In the previous *JAX for the impatient* notebook, we finished up with a linear regression example. As we know, linear regression can also be written as a single dense neural network layer, which we will show in the following so that we can compare how it's done.

A dense layer is a layer that has a kernel parameter $W\in\mathcal{M}_{m,n}(\mathbb{R})$ where $m$ is the number of features as an output of the model, and $n$ the dimensionality of the input, and a bias parameter $b\in\mathbb{R}^m$. The dense layers returns $Wx+b$ from an input $x\in\mathbb{R}^n$.

This dense layer is already provided by Flax in the `flax.linen` module (here imported as `nn`).

In [19]:
# We create one dense layer instance (taking 'features' parameter as input)
model = nn.Dense(features=5)

# features are num_output_features, something like what we defined dense in keras

Layers (and models in general, we'll use that word from now on) are subclasses of the `linen.Module` class.

### Model parameters & initialization

Parameters are not stored with the models themselves. You need to initialize parameters by calling the `init` function, using a PRNGKey and a dummy input parameter.

In [20]:
key1, key2 = random.split(random.PRNGKey(0))

In [21]:
x = random.normal(key1,(10,))
x

DeviceArray([-2.6105583 ,  0.03385283,  1.0863333 , -1.4802988 ,
              0.48895672,  1.062516  ,  0.54174834,  0.0170228 ,
              0.2722685 ,  0.30522448], dtype=float32)

#### Parameters initialization

In [22]:
params = model.init(key2, x)

In [23]:
params['params']['kernel'].shape

(10, 5)

In [24]:
params['params']['bias'].shape

(5,)

Apply a function on the tensor, according to the embedded structure

In [25]:
jax.tree_map(lambda x:x.shape, params)

FrozenDict({
    params: {
        bias: (5,),
        kernel: (10, 5),
    },
})

In [26]:
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (10,)) # Dummy input
params = model.init(key2, x) # Initialization call
jax.tree_map(lambda x: x.shape, params) # Checking output shapes

FrozenDict({
    params: {
        bias: (5,),
        kernel: (10, 5),
    },
})

*Note: JAX and Flax, like NumPy, are row-based systems, meaning that vectors are represented as row vectors and not column vectors. This can be seen in the shape of the kernel here.*

The result is what we expect: bias and kernel parameters of the correct size. Under the hood:

*   The dummy input variable `x` is used to trigger shape inference: we only declared the number of features we wanted in the output of the model, not the size of the input. Flax finds out by itself the correct size of the kernel.
*   The random PRNG key is used to trigger the initialization functions (those have default values provided by the module here).
* Initialization functions are called to generate the initial set of parameters that the model will use. Those are functions that take as arguments `(PRNG Key, shape, dtype)` and return an Array of shape `shape`.
* The init function returns the initialized set of parameters (you can also get the output of the evaluation on the dummy input with the same syntax but using the `init_with_output` method instead of `init`.

We see in the output that parameters are stored in a `FrozenDict` instance which helps deal with the functional nature of JAX by preventing any mutation of the underlying dict and making the user aware of it. Read more about it in the Flax docs. As a consequence, the following doesn't work:


In [27]:
try:
    params['new_key'] = jnp.ones((2,2))
except ValueError as e:
    print("Error: ", e)

Error:  FrozenDict is immutable.


To evaluate the model with a given set of parameters (never stored with the model), we just use the `apply` method by providing it the parameters to use as well as the input:

In [29]:
model.apply(params, x)

DeviceArray([-1.3721193 ,  0.61131495,  0.6442836 ,  2.2192965 ,
             -1.1271116 ], dtype=float32)

### Gradient descent

If you jumped here directly without going through the JAX part, here is the linear regression formulation we're going to use: from a set of data points $\{(x_i,y_i), i\in \{1,\ldots, k\}, x_i\in\mathbb{R}^n,y_i\in\mathbb{R}^m\}$, we try to find a set of parameters $W\in \mathcal{M}_{m,n}(\mathbb{R}), b\in\mathbb{R}^m$ such that the function $f_{W,b}(x)=Wx+b$ minimizes the mean squared error:
$$\mathcal{L}(W,b)\rightarrow\frac{1}{k}\sum_{i=1}^{k} \frac{1}{2}\|y_i-f_{W,b}(x_i)\|^2_2$$

Here, we see that the tuple $(W,b)$ matches the parameters of the Dense layer. We'll perform gradient descent using those. Let's first generate the fake data we'll use. The data is exactly the same as in the JAX part's linear regression pytree example.

### Create fake data

In [60]:
# set problem dimensions
n_samples = 20
x_dim = 10
y_dim = 5

Generate random ground truth

In [61]:
key = random.PRNGKey(12)
k1, k2 = random.split(key)

W = random.normal(k1, (x_dim, y_dim))
b = random.normal(k2, (y_dim, ))

Store the parameters in a pytree(freezed)

In [62]:
true_params = freeze({'params':{'bias': b, 'kernel': W}})

Generate samples with additional noise

In [63]:
key_sample, key_noise = random.split(k1)

x_samples = random.normal(key_sample, (n_samples, x_dim))
y_samples = jnp.dot(x_samples, W) + b + 0.1*random.normal(key_noise,(n_samples, y_dim))

#### To summarize

In [67]:
def get_problem(seed):
    """
    Fetch a problem to solve 
    """
    key = random.PRNGKey(seed)
    k1, k2 = random.split(key)

    W = random.normal(k1, (x_dim, y_dim))
    b = random.normal(k2, (y_dim, ))
    true_params = freeze({'params':{'bias': b, 'kernel': W}})
    
    key_sample, key_noise = random.split(k1)

    x_samples = random.normal(key_sample, (n_samples, x_dim))
    y_samples = jnp.dot(x_samples, W) + b + 0.1*random.normal(key_noise,(n_samples, y_dim))
    return W, b, true_params, x_samples, y_samples

In [68]:
W, b, true_params, x_samples, y_samples = get_problem(42)

Check the data **dimensions** with tree_map

In [64]:
jax.tree_map(lambda x:x.shape,{"x_samples":x_samples,"y_samples":y_samples})

{'x_samples': (20, 10), 'y_samples': (20, 5)}

We copy the same training loop that we used in the JAX pytree linear regression example with `jax.value_and_grad()`, but here we can use `model.apply()` instead of having to define our own feed-forward function (`predict_pytree()` in the JAX example).

In [49]:
def mse(params, x_batched, y_batched):
    def squared_error(x,y):
        pred = model.apply(params, x)
        return jnp.inner(y-pred, y-pred) / 2.0
    # vmap is to vectorize the squared_error operation
    return jnp.mean(jax.vmap(squared_error)(x_batched, y_batched),axis=0)

In [50]:
learning_rate = 0.3
print('Loss for "true" W,b:\t', mse(true_params, x_samples, y_samples))
loss_grad_fn = jax.value_and_grad(mse)

Loss for "true" W,b:	 0.027116928


### An update function with a lambda optimizer

In this case ```tree_map``` function takes 1 lambda function and 2 arguements(2 pytree with same shape)

Hence the lambda function takes 2 arguements.

In [51]:
@jax.jit
def update_params(params, learning_rate, grads):
    """
    Pretty much a SGD expressed in a lambda
    """
    params = jax.tree_map(
        lambda parameters,gradients: parameters- learning_rate * gradients, params, grads
    )
    return params

### Training

In [52]:
for i in range(101):
    # Perform on gradient upate
    loss_val, grads = loss_grad_fn(params, x_samples, y_samples)
    params = update_params(params, learning_rate, grads)
    if i%10 == 0:
        print(f'Loss step {i}: ', loss_val)

Loss step 0:  27.911596
Loss step 10:  0.52542466
Loss step 20:  0.19694467
Loss step 30:  0.104476385
Loss step 40:  0.06338705
Loss step 50:  0.041814588
Loss step 60:  0.0295865
Loss step 70:  0.022402437
Loss step 80:  0.018111883
Loss step 90:  0.015530384
Loss step 100:  0.013971983


And finally perform the gradient descent.

### Optimizing with Optax

Flax used to use its own `flax.optim` package for optimization, but with
[FLIP #1009](https://github.com/google/flax/blob/main/docs/flip/1009-optimizer-api.md)
this was deprecated in favor of
[Optax](https://github.com/deepmind/optax).

Basic usage of Optax is straightforward:

1.   Choose an optimization method (e.g. `optax.sgd`).
2.   Create optimizer state from parameters.
3.   Compute the gradients of your loss with `jax.value_and_grad()`.
4.   At every iteration, call the Optax `update` function to update the internal
     optimizer state and create an update to the parameters. Then add the update
     to the parameters with Optax's `apply_updates` method.

Note that Optax can do a lot more: it's designed for composing simple gradient
transformations into more complex transformations that allows to implement a
wide range of optimizers. There is also support for changing optimizer
hyperparameters over time ("schedules"), applying different updates to different
parts of the parameter tree ("masking") and much more. For details please refer
to the
[official documentation](https://optax.readthedocs.io/en/latest/).

In [75]:
import optax

W, b, true_params, x_samples, y_samples = get_problem(42)
alpha = .1

# initialize the optimizers, this part is like pytorch,
# with learning rate with model parameters

# tx = optax.sgd(learning_rate=alpha)
tx = optax.adam(learning_rate=alpha) # adam optimizer
opt_state = tx.init(params)

loss_grad_fn = jax.value_and_grad(mse)

In [76]:
for i in range(101):
    loss_val, grads = loss_grad_fn(params, x_samples, y_samples)
    # optimizer state was updated all the while
    updates, opt_state = tx.update(grads, opt_state)
    # apply the updates to the parameters
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        print(f"Loss step {i}: {loss_val}")

Loss step 0: 0.11747979372739792
Loss step 10: 0.025752363726496696
Loss step 20: 0.017380256205797195
Loss step 30: 0.016398658975958824
Loss step 40: 0.013886086642742157
Loss step 50: 0.013021932914853096
Loss step 60: 0.012843652628362179
Loss step 70: 0.012776580639183521
Loss step 80: 0.012747618369758129
Loss step 90: 0.012733488343656063
Loss step 100: 0.012730260379612446


### Serializing the result

Now that we're happy with the result of our training, we might want to save the model parameters to load them back later. Flax provides a serialization package to enable you to do that.

In [77]:
from flax import serialization

In [82]:
bytes_output = serialization.to_bytes(params)
bytes_output

b'\x81\xa6params\x82\xa4bias\xc7!\x01\x93\x91\x05\xa7float32\xc4\x14\x96\xf8\x0b\xbf\xbc\xaa\x84?B\xb2?\xbfp\xb2\x05\xbf 3\x8f?\xa6kernel\xc7\xd6\x01\x93\x92\n\x05\xa7float32\xc4\xc8\x0bv\x0c@\xbc\x8d\x8b?\xcd\xcc\xa4\xbf\xa7K\\\xbe\xb8\x8d3?J\xa9\xd1\xbf^\x88m\xbf\x05\xb2\xca?\x12\xcb!?\'\x81i\xbf\x9b\x82X?\xa7\x1a\x1a@\x12\xd4\x19?\xd6X\xf6;\xf3\xd8\xbe\xbd\xd0\x91\x97\xbfKY\x82>\x84fY\xbf\x02\x02\x04\xc0\x05\xce\xea?\x9d\x03r?\xf98\x07?\xa0x\x9a<D5\x0f\xbeD\xc17\xc0\x04h*?\xf1r\x80\xbe\xbc\xb0\x13\xc0\x9c(\x92\xbe\xddW\x9c>\x8d\xfaY?ODi\xbe\xc8B\xed\xbev\xba,? \xe2\xcf>x/\xe1\xbe\xea\xe0\xb5\xbe\xa4\xc9#?\x8fu\xd4\xbe\xa1a\x1c\xbe\xa8U"\xc0\xc6\x8a\x97\xbd\xc6\x92\xb0?l\xe5G\xbf\xa8\x10\x95=\xa5\xac\xc5?\xb8\x8a\x1a?\x9c\xd0M\xbfY\x08\xf9?\xee!\x86?'

In [84]:
dict_output = serialization.to_state_dict(params)
dict_output

{'params': {'bias': DeviceArray([-0.5467619 ,  1.0364604 , -0.74881375, -0.522254  ,
                1.1187477 ], dtype=float32),
  'kernel': DeviceArray([[ 2.1947048 ,  1.0902629 , -1.2875    , -0.21513234,
                 0.7013812 ],
               [-1.6379788 , -0.92786205,  1.5835577 ,  0.63200486,
                -0.91212696],
               [ 0.8457429 ,  2.4078767 ,  0.6008922 ,  0.00751791,
                -0.09318724],
               [-1.1841373 ,  0.2545875 , -0.8492205 , -2.0626225 ,
                 1.8344122 ],
               [ 0.94536763,  0.5282131 ,  0.01885635, -0.13985163,
                -2.871171  ],
               [ 0.66564965, -0.25087693, -2.307662  , -0.28546607,
                 0.30535784],
               [ 0.85147935, -0.22779964, -0.46340013,  0.67472017,
                 0.40602207],
               [-0.43981528, -0.35523158,  0.63979554, -0.4149594 ,
                -0.15271617],
               [-2.536478  , -0.07399516,  1.3794792 , -0.78084445,
        

With same structure and messed up value

To load the model back, you'll need to use as a template the model parameter structure, like the one you would get from the model initialization. Here, we use the previously generated `params` as a template. Note that this will produce a new variable structure, and not mutate in-place.

*The point of enforcing structure through template is to avoid users issues downstream, so you need to first have the right model that generates the parameters structure.*

In [95]:
param_structure = jax.tree_map(lambda x:x*1.2,params)

serialization.from_bytes(param_structure,  bytes_output)

FrozenDict({
    params: {
        bias: array([-0.5467619 ,  1.0364604 , -0.74881375, -0.522254  ,  1.1187477 ],
              dtype=float32),
        kernel: array([[ 2.1947048 ,  1.0902629 , -1.2875    , -0.21513234,  0.7013812 ],
               [-1.6379788 , -0.92786205,  1.5835577 ,  0.63200486, -0.91212696],
               [ 0.8457429 ,  2.4078767 ,  0.6008922 ,  0.00751791, -0.09318724],
               [-1.1841373 ,  0.2545875 , -0.8492205 , -2.0626225 ,  1.8344122 ],
               [ 0.94536763,  0.5282131 ,  0.01885635, -0.13985163, -2.871171  ],
               [ 0.66564965, -0.25087693, -2.307662  , -0.28546607,  0.30535784],
               [ 0.85147935, -0.22779964, -0.46340013,  0.67472017,  0.40602207],
               [-0.43981528, -0.35523158,  0.63979554, -0.4149594 , -0.15271617],
               [-2.536478  , -0.07399516,  1.3794792 , -0.78084445,  0.07278568],
               [ 1.5443312 ,  0.6036792 , -0.8039644 ,  1.9455673 ,  1.0479105 ]],
              dtype=float32

## Defining your own models

Flax allows you to define your own models, which should be a bit more complicated than a linear regression. In this section, we'll show you how to build simple models. To do so, you'll need to create subclasses of the base `nn.Module` class.

*Keep in mind that we imported* `linen as nn` *and this only works with the new linen API*

### Module basics

The base abstraction for models is the `nn.Module` class, and every type of predefined layers in Flax (like the previous `Dense`) is a subclass of `nn.Module`. Let's take a look and start by defining a simple but custom multi-layer perceptron i.e. a sequence of Dense layers interleaved with calls to a non-linear activation function.

In [101]:
class ExplicitMLP(nn.Module):
    features: Sequence[int]
    
    def setup(self):
        self.layers = [nn.Dense(feat)
                      for feat in self.features]
        
    def __call__(self, inputs):
        x = inputs
        for i, lyr in enumerate(self.layers):
            x = lyr(x)
            if i!=len(self.layers)-1:
                x = nn.relu(x)
        return x
    
key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (4,4))

model = ExplicitMLP(features=[3,4,5])
params = model.init(key2, x)
y = model.apply(params, x)

# print out the mdoel structure
jax.tree_map(lambda x:x.shape, params)

FrozenDict({
    params: {
        layers_0: {
            bias: (3,),
            kernel: (4, 3),
        },
        layers_1: {
            bias: (4,),
            kernel: (3, 4),
        },
        layers_2: {
            bias: (5,),
            kernel: (4, 5),
        },
    },
})

As we can see, a `nn.Module` subclass is made of:

*   A collection of data fields (`nn.Module` are Python dataclasses) - here we only have the `features` field of type `Sequence[int]`.
*   A `setup()` method that is being called at the end of the `__postinit__` where you can register submodules, variables, parameters you will need in your model.
*   A `__call__` function that returns the output of the model from a given input.
*   The model structure defines a pytree of parameters following the same tree structure as the model: the params tree contains one `layers_n` sub dict per layer, and each of those contain the parameters of the associated Dense layer. The layout is very explicit.

*Note: lists are mostly managed as you would expect (WIP), there are corner cases you should be aware of as pointed out* [here](https://github.com/google/flax/issues/524)

Since the module structure and its parameters are not tied to each other, you can't directly call `model(x)` on a given input as it will return an error. The `__call__` function is being wrapped up in the `apply` one, which is the one to call on an input:

In [102]:
try:
    y = model(x) # Returns an error
except AttributeError as e:
    print(e)

"ExplicitMLP" object has no attribute "layers"


Since here we have a very simple model, we could have used an alternative (but equivalent) way of declaring the submodules inline in the `__call__` using the `@nn.compact` annotation like so:

In [106]:
class SimpleMLP(nn.Module):
    features: Sequence[int]
        
    @nn.compact
    def __call__(self, inputs):
        x = inputs
        for i, feat in enumerate(self.features):
            x = nn.Dense(feat, name=f'layer_{i}')(x)
            if i!=len(self.features)-1:
                x = nn.relu(x)
        return x

In [107]:
key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (2,2))

model = SimpleMLP(features=[4,5,6])
params = model.init(key2, x)
y = model.apply(params, x)

In [109]:
jax.tree_map(lambda x:x.shape, params)

FrozenDict({
    params: {
        layer_0: {
            bias: (4,),
            kernel: (2, 4),
        },
        layer_1: {
            bias: (5,),
            kernel: (4, 5),
        },
        layer_2: {
            bias: (6,),
            kernel: (5, 6),
        },
    },
})

There are, however, a few differences you should be aware of between the two declaration modes:

*   In `setup`, you are able to name some sublayers and keep them around for further use (e.g. encoder/decoder methods in autoencoders).
*   If you want to have multiple methods, then you **need** to declare the module using `setup`, as the `@nn.compact` annotation only allows one method to be annotated.
*   The last initialization will be handled differently. See these notes for more details (TODO: add notes link).


### Module parameters

In the previous MLP example, we relied only on predefined layers and operators (`Dense`, `relu`). Let's imagine that you didn't have a Dense layer provided by Flax and you wanted to write it on your own. Here is what it would look like using the `@nn.compact` way to declare a new modules:

In [111]:
class SimpleDense(nn.Module):
    features: int
    kernel_init: Callable = nn.initializers.lecun_normal()
    bias_init: Callable = nn.initializers.zeros
        
    @nn.compact
    def __call__(self, inputs):
        kernel = self.param('kernel',
                            self.kernel_init,
                            (inputs.shape[-1], self.features)
                           )
        y = lax.dot_general(inputs, kernel, (((inputs.ndim - 1,),(0,)),((), ())),)
        
        bias = self.param('bias', self.bias_init, (self.features,))
        y = y + bias
        return y
    
key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (4,4))

model = SimpleDense(features=3)
params = model.init(key2, x)
y = model.apply(params, x)

jax.tree_map(lambda x:x.shape, params)

FrozenDict({
    params: {
        bias: (3,),
        kernel: (4, 3),
    },
})

Here, we see how to both declare and assign a parameter to the model using the `self.param` method. It takes as input `(name, init_fn, *init_args)` : 

*   `name` is simply the name of the parameter that will end up in the parameter structure.
*   `init_fn` is a function with input `(PRNGKey, *init_args)` returning an Array, with `init_args` being the arguments needed to call the initialisation function.
*   `init_args` are the arguments to provide to the initialization function.

Such params can also be declared in the `setup` method; it won't be able to use shape inference because Flax is using lazy initialization at the first call site.

### Variables and collections of variables

As we've seen so far, working with models means working with:

*   A subclass of `nn.Module`;
*   A pytree of parameters for the model (typically from `model.init()`);

However this is not enough to cover everything that we would need for machine learning, especially neural networks. In some cases, you might want your neural network to keep track of some internal state while it runs (e.g. batch normalization layers). There is a way to declare variables beyond the parameters of the model with the `variable` method.

For demonstration purposes, we'll implement a simplified but similar mechanism to batch normalization: we'll store running averages and subtract those to the input at training time. For proper batchnorm, you should use (and look at) the implementation [here](https://github.com/google/flax/blob/main/flax/linen/normalization.py).

In [22]:
class BiasAdderWithRunningMean(nn.Module):
  decay: float = 0.99

  @nn.compact
  def __call__(self, x):
    # easy pattern to detect if we're initializing via empty variable tree
    is_initialized = self.has_variable('batch_stats', 'mean')
    ra_mean = self.variable('batch_stats', 'mean',
                            lambda s: jnp.zeros(s),
                            x.shape[1:])
    mean = ra_mean.value # This will either get the value or trigger init
    bias = self.param('bias', lambda rng, shape: jnp.zeros(shape), x.shape[1:])
    if is_initialized:
      ra_mean.value = self.decay * ra_mean.value + (1.0 - self.decay) * jnp.mean(x, axis=0, keepdims=True)

    return x - ra_mean.value + bias


key1, key2 = random.split(random.PRNGKey(0), 2)
x = jnp.ones((10,5))
model = BiasAdderWithRunningMean()
variables = model.init(key1, x)
print('initialized variables:\n', variables)
y, updated_state = model.apply(variables, x, mutable=['batch_stats'])
print('updated state:\n', updated_state)

initialized variables:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([0., 0., 0., 0., 0.], dtype=float32),
    },
    params: {
        bias: DeviceArray([0., 0., 0., 0., 0.], dtype=float32),
    },
})
updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.01, 0.01, 0.01, 0.01, 0.01]], dtype=float32),
    },
})


Here, `updated_state` returns only the state variables that are being mutated by the model while applying it on data. To update the variables and get the new parameters of the model, we can use the following pattern:

In [23]:
for val in [1.0, 2.0, 3.0]:
  x = val * jnp.ones((10,5))
  y, updated_state = model.apply(variables, x, mutable=['batch_stats'])
  old_state, params = variables.pop('params')
  variables = freeze({'params': params, **updated_state})
  print('updated state:\n', updated_state) # Shows only the mutable part

updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.01, 0.01, 0.01, 0.01, 0.01]], dtype=float32),
    },
})
updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.0299, 0.0299, 0.0299, 0.0299, 0.0299]], dtype=float32),
    },
})
updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.059601, 0.059601, 0.059601, 0.059601, 0.059601]], dtype=float32),
    },
})


From this simplified example, you should be able to derive a full BatchNorm implementation, or any layer involving a state. To finish, let's add an optimizer to see how to play with both parameters updated by an optimizer and state variables.

*This example isn't doing anything and is only for demonstration purposes.*

In [29]:
def update_step(tx, apply_fn, x, opt_state, params, state):

  def loss(params):
    y, updated_state = apply_fn({'params': params, **state},
                                x, mutable=list(state.keys()))
    l = ((x - y) ** 2).sum()
    return l, updated_state

  (l, state), grads = jax.value_and_grad(loss, has_aux=True)(params)
  updates, opt_state = tx.update(grads, opt_state)
  params = optax.apply_updates(params, updates)
  return opt_state, params, state

x = jnp.ones((10,5))
variables = model.init(random.PRNGKey(0), x)
state, params = variables.pop('params')
del variables
tx = optax.sgd(learning_rate=0.02)
opt_state = tx.init(params)

for _ in range(3):
  opt_state, params, state = update_step(tx, model.apply, x, opt_state, params, state)
  print('Updated state: ', state)

Updated state:  FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.01, 0.01, 0.01, 0.01, 0.01]], dtype=float32),
    },
})
Updated state:  FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.0199, 0.0199, 0.0199, 0.0199, 0.0199]], dtype=float32),
    },
})
Updated state:  FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.029701, 0.029701, 0.029701, 0.029701, 0.029701]], dtype=float32),
    },
})


Note that the above function has a quite verbose signature and it would not actually
work with `jax.jit()` because the function arguments are not "valid JAX types".

We provide a handy wrapper that simplifies the above code, see:

https://flax.readthedocs.io/en/latest/flax.training.html#train-state

### Exporting to Tensorflow's SavedModel with jax2tf

JAX released an experimental converter called [jax2tf](https://github.com/google/jax/tree/main/jax/experimental/jax2tf), which allows converting trained Flax models into Tensorflow's SavedModel format (so it can be used for [TF Hub](https://www.tensorflow.org/hub), [TF.lite](https://www.tensorflow.org/lite), [TF.js](https://www.tensorflow.org/js), or other downstream applications). The repository contains more documentation and has various examples for Flax.